In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from CDAE import AutoEncoder
from tqdm import trange
from utils import *
from sklearn.cluster import KMeans, spectral_clustering
from sklearn.decomposition import PCA

/Users/larry/Py3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Data preprocessing

In [2]:
import math
def get_map(list_):
    map_ = {}
    for idx, ident in enumerate(list_):
        map_[ident] = idx
        
    return map_

def get_matrix(data):
    matrix = np.zeros((total_usr, total_item), dtype=np.float32)
    for line in data:
        uid = user_map[line[0]]
        iid = item_map[line[1]]
        matrix[uid, iid] = 1
    
    return matrix

userList = np.load('../data/netflix/netflix_userList.npy')
itemList = np.load('../data/netflix/netflix_itemList.npy')

total_usr = len(userList)
total_item = len(itemList)

user_map = get_map(userList)
item_map = get_map(itemList)

train_rating_all = np.load('../data/netflix/train_rating_all.npy')

import pickle

train_indices_all = None
test_indices_all = None

with open('../data/netflix/train_indices_all.pkl', 'rb') as train_indice_file:
    train_indices_all = pickle.load(train_indice_file)
    train_indice_file.close()
    
with open('../data/netflix/test_indices_all.pkl', 'rb') as test_indice_file:
    test_indices_all = pickle.load(test_indice_file)
    test_indice_file.close()

In [3]:
df = pd.read_csv('../data/itri/rating_itri_pruned.csv')
df['freq'] = df.groupby('uid')['uid'].transform('count')  # count frequncy by column's values
df = df[df['freq'] > 5]  # remove row which corresponding frequence < 5
df_array = df.as_matrix()

userList = sorted(df['uid'].unique())
itemList = sorted(df['iid'].unique())

total_usr = len(df['uid'].unique())
total_item = len(df['iid'].unique())

user_map = get_map(userList)
item_map = get_map(itemList)
    

sparsity = len(df)/(total_usr*total_item)
print("sparsity of ratings is %.2f%%" %(sparsity*100))
print ("num. of users: %d, num. of items: %d, num. of ratings: %d" % (total_usr, total_item, len(df)))

sparsity of ratings is 5.16%
num. of users: 276, num. of items: 989, num. of ratings: 14072


# Clustering

In [ ]:
user_vectors = np.load('../data/itri/feature_vectors.npy')
pca = PCA(n_components=2, svd_solver='full')
pca_out = pca.fit_transform(user_vectors)
colors = np.random.rand(len(pca_out[:, 0]))


plt.scatter(pca_out[:, 0], pca_out[:, 1], c=colors, alpha=0.5)
plt.savefig('figs/feature_vecPCA_scatter.jpg')
plt.gcf().clear()

In [ ]:
user_vectors = np.load('../data/netflix/feature_vectors.npy')
pca = PCA(n_components=10, svd_solver='full')
pca_out = pca.fit_transform(user_vectors)
NUM_CLUSTER = 50
kmeans = KMeans(n_clusters=NUM_CLUSTER, n_init=10, algorithm='full')


kmeans.fit(pca_out)

In [ ]:
label_map = {}
for i in kmeans.labels_:
    if i not in label_map:
        label_map[i] = 1
    else:
        label_map[i] += 1

print (label_map)

In [ ]:
label_index = {}
for i in range(NUM_CLUSTER):
    label_index[i] = []
    
label_list = list(kmeans.labels_)

for idx, i in enumerate(label_list):
    label_index[i].append(idx)

rating = np.zeros((total_usr, total_item), dtype=np.int8)
for line in df_array:
    uid = user_map[line[0]]
    iid = item_map[line[1]]
    rating[uid, iid] = 1

In [ ]:
test_aps_5 = []
test_aps_10 = []
test_rec_5 = []
test_rec_10 = []

for i in range(NUM_CLUSTER):
    print ("Cluster %d." % (i))
    train_rating = np.take(train_rating_all, label_index[i], axis=0)
    train_user = label_index[i]
    # train_rating, train_indices, test_indices = gen_train_test(rating_n)
    # train_rating = np.take(rating, label_index[i], axis=0)
    train_indices = np.take(train_indices_all, label_index[i])
    test_indices = np.take(test_indices_all, label_index[i])
    
    tf.reset_default_graph()

    autoencoder = AutoEncoder(user_num=total_usr, item_num=total_item, mode='user', loss_function='log_loss',
                              batch_size=64, epochs=500)
    autoencoder.model_load(0)
    autoencoder.train(rating=train_rating,
                      train_idents=train_user,
                      train_indices=train_indices,
                      test_indices=test_indices)
    
    """autoencoder.train(rating=rating,
                     train_idents=train_user)"""
    
    test_ap_5 = autoencoder.log['ap@5']
    test_ap_10 = autoencoder.log['ap@10']
    recs_5 = autoencoder.log['recall@5']
    recs_10 = autoencoder.log['recall@10']
    
    # top_N_ap = sorted(test_ap,reverse=True)[:20]
    # test_aps.append(np.mean(top_N_ap))
    
    test_aps_5.append(max(test_ap_5))
    test_aps_10.append(max(test_ap_10))
    test_rec_5.append(max(recs_5))
    test_rec_10.append(max(recs_10))
    
    plt.plot(range(len(test_ap_10)), test_ap_10, color='green', label='Test AP')
    # plt.plot(range(len(test_loss)), test_loss, color='red', label='Test loss')
    plt.legend(loc="upper right")
    plt.title("Train 200 epoch")
    plt.xlabel('#Epoch')
    plt.ylabel('AP_10')
    plt.savefig('./figs/test_ap_cluster_%d.jpg' % (i))
    plt.gcf().clear()
    

In [ ]:
ap_5 = 0

for i in range(NUM_CLUSTER):
    num = label_map[i]
    
    ap_5 += test_aps_5[i] * num
    print ("Cluster %d, aps: %f, num: %d, weighted ap: %f" % (i, test_aps_5[i], num, test_aps_5[i]*num))
    
ap_5 = ap_5 / (total_usr)

print ("Over all average ap: %f" % (ap_5))

In [ ]:
ap_10 = 0

for i in range(NUM_CLUSTER):
    num = label_map[i]
    
    ap_10 += test_aps_10[i] * num
    print ("Cluster %d, aps: %f, num: %d, weighted ap: %f" % (i, test_aps_10[i], num, test_aps_10[i]*num))
    
ap_10 = ap_10 / (total_usr)

print ("Over all average ap: %f" % (ap_10))

In [ ]:
recall_5 = 0

for i in range(NUM_CLUSTER):
    num = label_map[i]
    
    recall_5 += test_rec_5[i] * num
    print ("Cluster %d, aps: %f, num: %d, weighted ap: %f" % (i, test_rec_5[i], num, test_rec_5[i]*num))
    
recall_5 = recall_5 / (total_usr)

print ("Over all average ap: %f" % (recall_5))

In [ ]:
recall_10 = 0

for i in range(NUM_CLUSTER):
    num = label_map[i]
    
    recall_10 += test_rec_10[i] * num
    print ("Cluster %d, aps: %f, num: %d, weighted ap: %f" % (i, test_rec_10[i], num, test_rec_10[i]*num))
    
recall_10 = recall_10 / (total_usr)

print ("Over all average ap: %f" % (recall_10))

# Old

### For netflix huge dataset

user_train_rating = np.load('../data/netflix/rating_matrix_CDAE.npy')

train_rating_all, train_indices_all, test_indices_all = gen_train_test(user_train_rating)

import pickle

np.save('../data/netflix/train_rating_all.npy', train_rating_all)

with open('../data/netflix/train_indices_all.pkl', 'wb') as train_indice_file:
    pickle.dump(train_indices_all, train_indice_file)
    
with open('../data/netflix/test_indices_all.pkl', 'wb') as test_indice_file:
    pickle.dump(test_indices_all, test_indice_file)

### For others

In [4]:
train_data = df.as_matrix()

user_train_rating = np.zeros((total_usr, total_item), dtype=np.int32)

for line in train_data:
    uid = user_map[line[0]]
    iid = item_map[line[1]]
    user_train_rating[uid, iid] = 1

In [5]:
topN = np.count_nonzero(user_train_rating, axis=0).argsort()[::-1][:10]
others = [k for k in range(total_item) if k not in topN]

## Build Model and Training

In [6]:
def train_train(inputs, train_user_all, train_indices, test_indices, topN=None, weight=None):
    logs = {'ap@5': [], 'ap@10': [], 'ap@5_top': [], 'ap@10_top': [], 'filter_loss': []}
    
    if topN is not None:
        filter_ = np.ones(total_item, dtype=np.int32)
        for i in range(filter_.shape[0]):
            if i in topN:
                filter_[i] = weight
    else:
        filter_ = np.ones(total_item, dtype=np.int8)
        
    train_idents_idx = [k for k in range(len(train_user_all))]
        
    for epoch in trange(500):
        # total_loss = 0
        ap_at_5 = []
        ap_at_5_top = []
        ap_at_10 = []
        ap_at_10_top = []
        
        for n in train_user_all:
            input_ = np.take(inputs, train_idents_idx[n:n+1], axis=0)
            target_ = input_
            idents_ = train_user_all[n:n+1]
            
            recon, filter_loss, _ = autoencoder.sess.run(
                [autoencoder.decode, autoencoder.filter_loss, autoencoder.optim],
                feed_dict={
                    autoencoder.input: input_,
                    autoencoder.target: target_,
                    autoencoder.ident: idents_,
                    autoencoder.filter: filter_,
                })
            
            top10 = get_topN(recon, train_indices[n:n+1], N=10)
            top5 = get_topN(recon, train_indices[n:n+1], N=5)
            
            top10_top = np.asarray([[k for k in test_indices[n] if k in topN]])
            top5_top = np.asarray([[k for k in  test_indices[n] if k in topN]])
            
            iAP_5 = avg_precision(top5, test_indices[n:n+1])
            iAP_10 = avg_precision(top10, test_indices[n:n+1])
            iAP_5_top = avg_precision(top5, top5_top)
            iAP_10_top = avg_precision(top10, top10_top)
            
            if iAP_5 is not None:
                ap_at_5.append(iAP_5)  
            if iAP_10 is not None:
                ap_at_10.append(iAP_10)
            if iAP_5_top is not None:
                ap_at_5_top.append(iAP_5_top)
            if iAP_10_top is not None:
                ap_at_10_top.append(iAP_10_top)
                
        logs['ap@5'].append(sum(ap_at_5)/len(ap_at_5))
        logs['ap@10'].append(sum(ap_at_10)/len(ap_at_10))
        logs['ap@5_top'].append(sum(ap_at_5_top)/len(ap_at_5_top))
        logs['ap@10_top'].append(sum(ap_at_10_top)/len(ap_at_10_top))
        logs['filter_loss'].append(filter_loss)
        
    logs['filter_loss'] = np.asarray(logs['filter_loss'])
        
    return logs

In [7]:
weights = [1, 10, 100, 1000]

for s in weights:
    train_rating_all, train_indices_all, test_indices_all = gen_train_test(user_train_rating)

    tf.reset_default_graph()

    train_user_all = np.nonzero(np.count_nonzero(train_rating_all, axis=1))[0]

    autoencoder = AutoEncoder(user_num=total_usr, item_num=total_item, mode='user', loss_function='cross_entropy',
                              batch_size=1, epochs=500)
    
    logs = train_train(train_rating_all, train_user_all, train_indices_all, test_indices_all, topN=topN, weight=s)
    
    test_ap = logs['ap@5']

    plt.plot(range((len(test_ap))), test_ap, color='green', label='Test AP')
    # plt.plot(range(len(test_loss)), test_loss, color='red', label='Test loss')
    plt.legend(loc="upper right")
    plt.title("test_ap@5_with_weight_%d.jpg'" % (s))
    plt.xlabel('#Epoch')
    plt.ylabel('AP')
    plt.savefig('./figs/test_ap@5_with_weight_%d.jpg' % (s))
    plt.gcf().clear()
    
    test_ap = logs['ap@10']

    plt.plot(range((len(test_ap))), test_ap, color='green', label='Test AP')
    # plt.plot(range(len(test_loss)), test_loss, color='red', label='Test loss')
    plt.legend(loc="upper right")
    plt.title("test_ap@10_with_weight_%d.jpg'" % (s))
    plt.xlabel('#Epoch')
    plt.ylabel('AP')
    plt.savefig('./figs/test_ap@10_with_weight_%d.jpg' % (s))
    plt.gcf().clear()
    
    test_ap = logs['ap@5_top']

    plt.plot(range((len(test_ap))), test_ap, color='green', label='Test AP')
    # plt.plot(range(len(test_loss)), test_loss, color='red', label='Test loss')
    plt.legend(loc="upper right")
    plt.title("test_ap@5_top_with_weight_%d.jpg'" % (s))
    plt.xlabel('#Epoch')
    plt.ylabel('AP')
    plt.savefig('./figs/test_ap@5_top_with_weight_%d.jpg' % (s))
    plt.gcf().clear()
    
    test_ap = logs['ap@10_top']

    plt.plot(range((len(test_ap))), test_ap, color='green', label='Test AP')
    # plt.plot(range(len(test_loss)), test_loss, color='red', label='Test loss')
    plt.legend(loc="upper right")
    plt.title("test_ap@10_top_with_weight_%d.jpg'" % (s))
    plt.xlabel('#Epoch')
    plt.ylabel('AP')
    plt.savefig('./figs/test_ap@10_top_with_weight_%d.jpg' % (s))
    plt.gcf().clear()
    
    topN_loss = []
    other_loss = []

    for i in logs['filter_loss']:
        topN_loss.append(sum(sum(np.take(i, topN, axis=1))))
        other_loss.append(sum(sum(np.take(i, others, axis=1))))
        
    plt.plot(range((len(topN_loss))), topN_loss, color='blue', label='topN Loss')
    plt.legend(loc="upper right")
    plt.title("topN_loss_with_weight_%d" % (s))
    plt.xlabel('#Epoch')
    plt.ylabel('cross entropy')
    plt.savefig('./figs/topN_loss_with_weight_%d.jpg' % (s))
    plt.gcf().clear()
    
    plt.plot(range((len(other_loss))), other_loss, color='blue', label='other Loss')
    plt.legend(loc="upper right")
    plt.title("other_loss_with_weight_%d" % (s))
    plt.xlabel('#Epoch')
    plt.ylabel('cross entropy')
    plt.savefig('./figs/other_loss_with_weight_%d.jpg' % (s))
    plt.gcf().clear()

100%|██████████| 500/500 [03:23<00:00,  2.46it/s]


## old

In [ ]:
train_rating_all, train_indices_all, test_indices_all = gen_train_test(user_train_rating)

tf.reset_default_graph()

train_user_all = np.nonzero(np.count_nonzero(train_rating_all, axis=1))[0]

autoencoder = AutoEncoder(user_num=total_usr, item_num=total_item, mode='user', loss_function='log_loss',
                          batch_size=1, epochs=200)


autoencoder.train(rating=train_rating_all,
                  train_idents=train_user_all,
                  train_indices=train_indices_all,
                  test_indices=test_indices_all,
                  topN=topN,
                  weight=100)

# autoencoder.model_save(0)

In [ ]:
train_loss = autoencoder.log['train_loss']

plt.plot(range(len(train_loss)), train_loss, color='blue', label='Train loss')
# plt.plot(range(len(test_loss)), test_loss, color='red', label='Test loss')
plt.legend(loc="upper right")
plt.xlabel('#Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
test_ap = autoencoder.log['ap@5']

plt.plot(range((len(test_ap))), test_ap, color='green', label='Test AP')
# plt.plot(range(len(test_loss)), test_loss, color='red', label='Test loss')
plt.legend(loc="upper right")
plt.title("Train 200 epoch")
plt.xlabel('#Epoch')
plt.ylabel('AP')
plt.show()

# print (np.mean(top_N_ap))

print (max(test_ap))

In [ ]:
test_recall = autoencoder.log['recall@10']

plt.plot(range(1000, (len(test_recall)+1)*1000, 1000), test_recall, color='green', label='Test Recall')
# plt.plot(range(len(test_loss)), test_loss, color='red', label='Test loss')
plt.legend(loc="upper right")
plt.title("Train 200 epoch")
plt.xlabel('#Epoch')
plt.ylabel('Recall')
plt.show()

# print (np.mean(top_N_ap))

print (max(test_recall))

In [ ]:
tf.reset_default_graph()
autoencoder = AutoEncoder(user_num=total_usr, item_num=total_item, mode='user', loss_function='cross_entropy',
                          batch_size=100, epochs=1000)
autoencoder.model_load(0)

In [ ]:
user_onehot_vectors = autoencoder.sess.run(autoencoder.vector_matrix)
u_vectors = np.load('../data/netflix/user_vectors.npy')

In [ ]:
feature_vectors = np.zeros((total_usr, 20), dtype=np.float32)
for i in range(total_usr):
    feature_vectors[i] = autoencoder.sess.run(
        autoencoder.code,
        feed_dict={
            autoencoder.input: [train_rating_all[i]],
            autoencoder.ident: [i]
        })
    
np.save('../data/netflix/feature_vectors.npy', feature_vectors)

In [ ]:
user_onehot_vectors = autoencoder.sess.run(autoencoder.vector_matrix)

np.save('../data/netflix/user_vectors.npy', user_onehot_vectors)